In [1]:
import os


In [9]:
folder='V:\\V_Drive\\DATA\\liver_binary/'
subdir=['train/','test/']
for subd in subdir:
    direct=['normal/','abnormal/']
    for dire in direct:
        os.makedirs(folder+subd+dire,exist_ok=True)
        

In [14]:
import shutil as s
import os
import random as r
r.seed(1)
val_ratio = 0.1
dire=['normal','abnormal']
for f in dire:
    src_dir='V:\\V_Drive\\DATA\\liver_binary\\binary_data_liver/'+f
    for file in os.listdir(src_dir):
        src=src_dir+'/'+file
        dest='V:\\V_Drive\\DATA\\liver_binary/train/'+f
        if r.random() < val_ratio:
            dest = 'V:\\V_Drive\\DATA\\liver_binary/test/'+f
        dst = dest +'/' +file
        s.copyfile(src, dst)
    

In [59]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rescale=1.0/255.0)
train_it=gen.flow_from_directory('V:\\V_Drive\\DATA\\liver_binary/train/',
		class_mode='categorical',classes=['abnormal','normal'], batch_size=5, target_size=(300, 300))

test_it = gen.flow_from_directory('V:\\V_Drive\\DATA\\liver_binary/test/',
		class_mode='categorical',classes=['abnormal','normal'], batch_size=5, target_size=(300, 300))

Found 136 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [60]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(300, 300, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(2, activation='softmax'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [61]:
model = define_model()

In [62]:
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=5, verbose=0)

_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)

print('> %.3f' % (acc * 100.0))

> 75.000


In [76]:
model.save('liver_mult.h5')

In [17]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
img=load_img('V:\\V_Drive\\DATA\\liver_binary\\binary_data_liver\\abnormal/grade2-001.jpg',target_size=(300, 300))
img=img_to_array(img)
img=img.reshape(1, 300, 300,3)
img = img.astype('float32')
img = img - [123.68, 116.779, 103.939]

In [18]:
result = model.predict(img)
if(result[0][0]>result[0][1]):
    print("Abnormal")
else:
      print("normal")

Abnormal


In [ ]:
import tensorflow as tf 
model=tf.keras.models.load_model('liver_mult.h5')
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()
file=open('liver_mtf.tflite','wb')
file.write(tflite_model)

In [21]:
import keras
model=keras.models.load_model('liver_mult.h5')

In [19]:
type(result)

numpy.ndarray

In [4]:
# Load TFLite model and allocate tensors.
import tensorflow as tf
interpreter = tf.lite.Interpreter(model_path="model.tflite") 
interpreter.allocate_tensors()
# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [7]:
import numpy as np
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.9999825]]
